## 各國家YLD量值隨年份變化
這是有關動畫如何呈現的程式碼，大致延沿襲以下網址的範例
https://plot.ly/python/gapminder-example

因為在動畫呈現上有許多的參數可以被調動，因而更詳盡的參數說明請見：https://plot.ly/python/reference

In [1]:
import plotly
import plotly.plotly as py
from plotly.grid_objs import Grid, Column
from plotly.tools import FigureFactory as FF 
from plotly.offline import init_notebook_mode, iplot
from IPython.display import display, HTML
import pandas as pd
import time
plotly.tools.set_credentials_file(username='user_name', api_key='your_keys')

In [2]:
dataset = pd.read_csv("breast_cancer.csv")
dataset["point"]=pd.Series([50]*len(dataset)) #讓資料點看起來比較大

In [3]:
years_from_col = set(dataset['year'])
years_ints = sorted(list(years_from_col))
years = [str(year) for year in years_ints] #得到所有的年份

# make list of continents
continents = []
for continent in dataset['continent']:
    if continent not in continents: 
        continents.append(continent)

columns = []
# make grid
for year in years:
    for continent in continents:
        dataset_by_year = dataset[dataset['year'] == int(year)]
        dataset_by_year_and_cont = dataset_by_year[dataset_by_year['continent'] == continent]
        for col_name in dataset_by_year_and_cont:
            # each column name is unique
            column_name = '{year}_{continent}_{header}_gapminder_grid'.format(
                year=year, continent=continent, header=col_name
            )
            a_column = Column(list(dataset_by_year_and_cont[col_name]), column_name)
            columns.append(a_column)

# upload grid
grid = Grid(columns)
url = py.grid_ops.upload(grid, 'gapminder_grid'+str(time.time()), auto_open=False)

In [4]:
figure = {  #先創建一個框架起來
    'data': [],
    'layout': {},
    'frames': [],
    'config': {'scrollzoom': True}
}

# fill in most of layout
figure['layout']['xaxis'] = {'range':[min(dataset.female),max(dataset.female)] ,'title': 'female', 'gridcolor': '#FFFFFF'}
figure['layout']['yaxis'] = {'title': 'male', 'gridcolor': '#FFFFFF'}
figure['layout']['hovermode'] = 'closest'
figure['layout']['plot_bgcolor'] = 'rgb(223, 232, 243)'
figure["layout"]["title"]="YLDs Compared with Countries (Origin Data)"

In [5]:
figure['layout']['sliders'] = { #設定在動畫呈現上的其它標題屬性
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'text-before-value-on-display',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': [...]
}

In [6]:
sliders_dict = {
    'active': 0,
    'yanchor': 'top',
    'xanchor': 'left',
    'currentvalue': {
        'font': {'size': 20},
        'prefix': 'Year:',
        'visible': True,
        'xanchor': 'right'
    },
    'transition': {'duration': 300, 'easing': 'cubic-in-out'},
    'pad': {'b': 10, 't': 50},
    'len': 0.9,
    'x': 0.1,
    'y': 0,
    'steps': []
}

In [7]:
figure['layout']['updatemenus'] = [ #關於按鈕的部份
    {
        'buttons': [
            {
                'args': [None, {'frame': {'duration': 500, 'redraw': False},
                         'fromcurrent': True, 'transition': {'duration': 300, 'easing': 'quadratic-in-out'}}],
                'label': 'Play',
                'method': 'animate'
            },
            {
                'args': [[None], {'frame': {'duration': 0, 'redraw': False}, 'mode': 'immediate',
                'transition': {'duration': 0}}],
                'label': 'Pause',
                'method': 'animate'
            }
        ],
        'direction': 'left',
        'pad': {'r': 10, 't': 87},
        'showactive': False,
        'type': 'buttons',
        'x': 0.1,
        'xanchor': 'right',
        'y': 0,
        'yanchor': 'top'
    }
]

custom_colors = { #對每一個類群的國家進行顏色的區別
    'Asia': 'rgb(220, 20, 60)',
    'Europe': 'rgb(255, 140, 0)',
    'Africa': 'rgb(169, 169, 169)',
    'Americas': 'rgb(60, 179, 113)',
    'Oceania': 'rgb(65, 105, 225)'
}

In [8]:
col_name_template = '{year}_{continent}_{header}_gapminder_grid'
year = 1952
for year in [1990]:
    for continent in continents:
        data_dict = {
            'xsrc': grid.get_column_reference(col_name_template.format(
                year=year, continent=continent, header='female'
            )),
            'ysrc': grid.get_column_reference(col_name_template.format(
                year=year, continent=continent, header='male'
            )),
            'mode': 'markers',
            'textsrc': grid.get_column_reference(col_name_template.format(
                year=year, continent=continent, header='country'
            )),
            'marker': {
                'sizemode': 'area',
                'sizeref': 0.4,
                'sizesrc': grid.get_column_reference(col_name_template.format(
                    year=year, continent=continent, header='point'
                )),
                'color': custom_colors[continent]
            },
            'name': continent
        }
        figure['data'].append(data_dict)


In [9]:
frame = {'data': [], 'name': "title"}
figure['layout']['sliders'] = [sliders_dict]

In [10]:
for year in years:
    frame = {'data': [], 'name': str(year)}
    for continent in continents:
        data_dict = {
            'xsrc': grid.get_column_reference(col_name_template.format(
                year=year, continent=continent, header='female'
            )),
            'ysrc': grid.get_column_reference(col_name_template.format(
                year=year, continent=continent, header='male'
            )),
            'mode': 'markers',
            'textsrc': grid.get_column_reference(col_name_template.format(
                year=year, continent=continent, header='country'
                )),
            'marker': {
                'sizemode': 'area',
                'sizeref': 0.4,
                'sizesrc': grid.get_column_reference(col_name_template.format(
                    year=year, continent=continent, header='point'
                )),
                'color': custom_colors[continent]
            },
            'name': continent
        }
        frame['data'].append(data_dict)

    figure['frames'].append(frame)
    slider_step = {'args': [
        [year],
        {'frame': {'duration': 300, 'redraw': False},
         'mode': 'immediate',
       'transition': {'duration': 300}}
     ],
     'label': year,
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

figure['layout']['sliders'] = [sliders_dict]

In [11]:
py.icreate_animations(figure, 'gapminder_example'+str(time.time()))